In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
print("complete")

/kaggle/input/cvalltogether/CV_brief.pdf
/kaggle/input/cvalltogether/drmperfect_cv-april-2020.pdf
/kaggle/input/cvalltogether/vita_external.pdf
/kaggle/input/cvalltogether/Klar CV.pdf
/kaggle/input/cvalltogether/blee_cv_2016.pdf
/kaggle/input/cvalltogether/Canales_Robert_CV.pdf
/kaggle/input/cvalltogether/MOORE-MONROY2015_0.pdf
/kaggle/input/cvalltogether/CURRICULUM-VITAE_DHG_012519.pdf
/kaggle/input/cvalltogether/RobertsonCV0818-2.pdf
/kaggle/input/cvalltogether/hameroff2016cv_0.pdf
/kaggle/input/cvalltogether/Alison-M-Meadow-cv.pdf
/kaggle/input/cvalltogether/agaspar_cv.pdf
/kaggle/input/cvalltogether/JO - 2171.pdf
/kaggle/input/cvalltogether/Hoit CV (4-11-16).pdf
/kaggle/input/cvalltogether/Liverman Selected CV May 2018.pdf
/kaggle/input/cvalltogether/LBarraza CV 2020.pdf
complete


In [2]:
!pip install PyPDF2
!pip install pdfplumber
print("complete")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.2/220.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 40.2 MB/s eta 0:00:00
complete


In [3]:
#all import
import pdfplumber
import pandas as pd
import os
import spacy
from spacy.matcher import Matcher

from datetime import datetime

import re
from nltk.corpus import stopwords

import PyPDF2
nlp = spacy.load("en_core_web_sm")

import csv

print("complete")

complete


In [4]:
# use nlp Ner to differentiate and divide the text data

def nerCV(personName,sectionName,sentDataFoeTest):
    allDataner=[]
    metadata001=[personName,sectionName+" details ",sectionName]
    countNo=0
    for train_dataText in sentDataFoeTest :
        
    
    
    
        doc = nlp(train_dataText)
        
        
        dldata=dict()
    
        for ent in doc.ents:
        
            if ent.label_.upper() in dldata:
                dldata[ent.label_.upper()].append(ent.text)
            else:
                dldata[ent.label_.upper()]=[ent.text]
            
    
        if dldata:
            countNo+=1
            
            for dl in dldata:
                metadata=metadata001.copy()
                if dl=='DATE':
                    for d in dldata[dl]:
                        if len(d)<3:
                            continue
                        metadata.append(sectionName+ " in "+str(dl)+" "+str(countNo)+" ")
                        metadata.append(d)
                        allDataner.append(metadata.copy())
                        #print(metadata)
                        metadata.pop()
                        metadata.pop()
                elif dl=='ORG':
                    for d in dldata[dl]:
                        if len(d)<3:
                            continue
                        metadata.append(sectionName+ " with "+str(dl)+" "+str(countNo)+" ")
                        metadata.append(d)
                        allDataner.append(metadata.copy())
                        metadata.pop()
                        metadata.pop()
                elif dl=='PERSON':
                    for d in dldata[dl]:
                        if len(d)<3:
                            continue
                        metadata.append(sectionName+ " associated with "+str(dl)+" "+str(countNo)+" ")
                        metadata.append(d)
                        allDataner.append(metadata.copy())
                        metadata.pop()
                        metadata.pop()
                elif dl=='GPE':
                    for d in dldata[dl]:
                        if len(d)<3:
                            continue
                        metadata.append(sectionName+ " from "+str(dl)+" "+str(countNo)+" ")
                        metadata.append(d)
                        allDataner.append(metadata.copy())
                        metadata.pop()
                        metadata.pop()
                else:
                    for d in dldata[dl]:
                        if len(d)<3:
                            continue
                        metadata.append(sectionName+ " in "+str(dl)+" "+str(countNo)+" ")
                        metadata.append(d)
                        allDataner.append(metadata.copy())
                        metadata.pop()
                        metadata.pop()
            #
            metadata.append("Original text "+str(countNo)+" ")
            metadata.append(train_dataText)
            allDataner.append(metadata.copy())
            metadata.pop()
            metadata.pop()
    
    return allDataner
print("complete")

complete


In [5]:
# extract person name from text data


def extract_name(resume_text):
    # load pre-trained model
    nlp = spacy.load('en_core_web_sm')

    # initialize matcher with a vocab
    matcher = Matcher(nlp.vocab)
    nlp_text = nlp(resume_text)
    
    # First name and Last name are always Proper Nouns
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
    
    matcher.add('NAME', [pattern], on_match = None)
    
    matches = matcher(nlp_text)
    res=[]
    
    for match_id, start, end in matches:
        span = nlp_text[start:end]
        temp=span.text
        if len(temp)>0:
            lastoption=temp
        if ("vitae" in temp.lower() or "sciences" in temp.lower() 
            or "engineering" in temp.lower() or "biographical" in temp.lower() 
            or "no" in temp.lower() or "title" in temp.lower() ):
            continue
        res.append(temp)
    
    return res
print("complete")

complete


In [6]:
# divide the text data into different parts or sections

def dfPartByPart(textList):
    

    reference=[ 
            #node1
            "articles","article","books","book","chapters","chapter","citations","citation","editorials","editorial","journals","journal",
               "seminars","seminar","scholarly",
            #node2
            "awards","award","proposals","proposal","grants","grant","honors","honor","scholarships","scholarship","sponsored",
            #node3
            "appointments","appointment","experiences","experience","services","service","employments","employment","practices","practice",
                "professionals","professional",
            #node4
            "affiliations","affiliation","memberships","membership","committees","committee",
            #node5
            "contacts","contact","introductions","introduction",
            #node6
            "publications","publication","conferences","conference","presentations","presentation","newsletters","newsletter","reports","report",
            #node7
            "educations","education","certificates","certificate","certifications","certification",
            #node8
            "researches","research",
            #node9
            "teaching","outreaches","outreach"
          ]

    
    
    
    collectedText=[]
    collectedByDictionary=dict()
    collectedParts=dict()
    keyDictionary="introductions"
    temp=[]

    

    for smallPartText in textList:
        if smallPartText==None:
            continue
   
        for partOfSmallPartText in smallPartText.split():
            if len(partOfSmallPartText)<2:
                continue
            result = list(filter(lambda x: x==partOfSmallPartText.lower(), reference))
            
            if result:
                
                collectedText.append(temp)
                if keyDictionary in collectedByDictionary:
                    collectedByDictionary[keyDictionary]=collectedByDictionary[keyDictionary]+temp
                else:
                    collectedByDictionary[keyDictionary]=temp
                keyDictionary=result[0]
                #print(result,keyDictionary)
                temp=[]
                break
        
        temp.append(smallPartText)
    
    
    collectedText.append(temp)
    if keyDictionary in collectedByDictionary:
        collectedByDictionary[keyDictionary]=collectedByDictionary[keyDictionary]+temp
    else:
        collectedByDictionary[keyDictionary]=temp

    return collectedByDictionary
print("complete")

complete


In [7]:
#read the data from pdf file and extract Ner data from it

def startProgram():
    
    #read the pdf file path
    
    folder_with_pdfs = '../input/cvalltogether'
    #linesOfFiles = []

    
    dataForTest=dict()

    listOfPdfFiles=[]

    alnerCVData=[]

    for pdf_file in os.listdir(folder_with_pdfs):
        if pdf_file.endswith('.pdf'):
        
        
            listOfPdfFiles.append(pdf_file)
            pdf_file_path = os.path.join(folder_with_pdfs, pdf_file)
        
            pdfFileObj = open(pdf_file_path,'rb')
            pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

# Getting number of pages in pdf file
            pages = pdfReader.numPages

            totalText=[]

# Loop for reading all the Pages
            for i in range(pages):

        # Creating a page object
                pageObj = pdfReader.getPage(i)
                text = pageObj.extractText().split("\n")
        
                totalText+=text
        
        

            pdfFileObj.close()
        
        
        
            newSectionDictionary=dfPartByPart(totalText)
        
            personName00=""
            personNameLast=""
            if 'contacts' in newSectionDictionary:
                text00=newSectionDictionary['contacts']
                personName00=extract_name(",".join(text00))
                personNameLast=personName00[0]
        
        
            if len(personName00)==0:
                if 'introductions' in newSectionDictionary:
                    text00=newSectionDictionary['introductions']
                    personName00=extract_name(",".join(text00))
            
                
                if len(personName00)==0:
                    personNameLast="no name"
                else:
             
            
                    personNameLast=personName00[0]
            else:
             
            
                personNameLast=personName00[0]
        
            alnerCVDataPart=[]
        
        
            dataForTest=newSectionDictionary
            for sectionDataSentTest in dataForTest:
                if len(dataForTest[sectionDataSentTest])>1:
                    nerret=nerCV(personNameLast,sectionDataSentTest,dataForTest[sectionDataSentTest][1:])
                
                    for i in nerret:
                        if i not in alnerCVDataPart:
                            alnerCVDataPart.append(i)
            alnerCVData.append(alnerCVDataPart)
            #break    
    return alnerCVData
print("complete")

complete


In [8]:
# transform the data into (Head,Relation,Tail) tiples 

def getCsvTriple(allNer):
    csvDataTriple=[["head","relation","tail"]]
    for listpart in allNer:
        
        i=0
        #print(listpart)
        n=len(listpart)
        while i+2<n:
            if i==n-1:
                break
            t=listpart[i:i+3].copy()
            if t not in csvDataTriple:
                csvDataTriple.append(t.copy())
            i+=2
            #pass
        #break
    return csvDataTriple
print("complete")

complete


In [9]:
#save the data in a .csv file format

def toCSVFile(dataTriple,noOfFile,nameOfFile):
    nameOfFile=nameOfFile+str(noOfFile)+".csv"
    with open(nameOfFile, 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)

    # write the header , already has a header
    #writer.writerow(header)

    # write multiple rows
        writer.writerows(dataTriple)
print("complete")

complete


In [10]:
#get the final data and save data into two types, 1st type is all the CV's 
#together, and 2nd type is for each of the CV's 

def totalControl():
    gtTest=startProgram()
    toCSVFile(gtTest,1,"AllCVNerData")
    for i in range(len(gtTest)):
        tempdata=getCsvTriple(gtTest[i])
        
        
        toCSVFile(tempdata,i+1,"partCVNerdata")
        
        
print("complete")

complete


In [11]:
# to start our program

totalControl()
print("complete")

complete
